# Overview

Randomly select 10 villages / wards in Ranchi and Khunti districts of Jharkhand and then select 3 points in each village / ward.


In [22]:
import geopandas as gpd
from pathlib import Path
import pandas as pd
import folium
from tqdm import tqdm
from shapely.geometry import LineString
tqdm.pandas()
from shapely import Point
from pin_drop_sampling2.utils import get_s2_cell_id, count_neighbors_in_radius, get_nearest_point_on_road, dist_in_meters

# set parameter for the number of PSUs to sample
NUM_PSUS_TO_SAMPLE = 10

# Set file locations

In [2]:
DB_DIR = Path.home() / 'IDinsight Dropbox' / 'Random Walk Testing' 
RAW_DATA = Path.home() / 'IDinsight Dropbox'/'Random Walk Testing'/'01_Raw data'
SHRUG_DIR = RAW_DATA / '02_Admin boundary data' / 'SHRUG' 
ROOFTOP_DIR = RAW_DATA / '01_Rooftop'/'India'
OUTPUT_DIR = DB_DIR / '03_Output' / '08_Jharkhand test pins'

# Import and clean files

In [3]:
# shrug
shrug = gpd.read_file(SHRUG_DIR / 'shrug-pc11-village-poly-shp' / 'village.shp')
shrug = shrug.rename(columns={'pc11_s_id': 'state_id', 'pc11_d_id': 'district_id', 'pc11_sd_id': 'subdistrict_id', 'pc11_tv_id': 'village_id'})
shrug['shrid2'] = '11-' + shrug['state_id'].astype(str) + '-' + shrug['district_id'].astype(str) + '-' + shrug['subdistrict_id'].astype(str) + '-' + shrug['village_id'].astype(str)
shrug = shrug.astype({'state_id': int, 'district_id': int, 'subdistrict_id': int, 'village_id': int})

# shrug district names and ids
shrug_districts = pd.read_csv(SHRUG_DIR / 'shrug-shrid-keys-csv' / 'shrug_district_names_ids.csv')

# shrug_loc_names 
shrug_loc_names = pd.read_csv(SHRUG_DIR / 'shrug-shrid-keys-csv' /  'shrid_loc_names.csv')
shrug_loc_names['urban'] = shrug_loc_names['town_name'].notnull().astype(int)
shrug_loc_names = shrug_loc_names[['shrid2', 'urban']]

# census
census = pd.read_csv(SHRUG_DIR/'shrug-pca11-csv'/'pc11_pca_clean_shrid.csv', usecols=['shrid2', 'pc11_pca_no_hh'])

# Filter SHRUG data for Ranchi and Khunti and merge with census data

In [4]:
# filter shrug data for Ranchi and Khunti districts
ranchi_khunti_districts =  shrug_districts[shrug_districts['district_name'].isin(['ranchi', 'khunti'])]
ranchi_khunti = shrug.merge(ranchi_khunti_districts, on=['state_id', 'district_id'])
print(f"length of ranchi khunti: {len(ranchi_khunti)}")

# left merge census and ranchi_khunti on shrid2
ranchi_khunti_census = ranchi_khunti.merge(census, on='shrid2', how='left')

# check how many shrug rows couldn't be merged with the census data
print(f"Num shrug rows with no census data: {int(ranchi_khunti_census['pc11_pca_no_hh'].isna().sum())}")

# merge with shrug_loc_names on shrid2 to get urban status of the sampled villages / wards
ranchi_khunti_census = ranchi_khunti_census.merge(shrug_loc_names, on='shrid2', how='left')

# check how many shrug rows couldn't be merged with the shrug_loc_names
print(f"Num shrug rows with no data on urban status: {int(ranchi_khunti_census['urban'].isna().sum())}")

# check the share of the population urban 
ranchi_khunti_census.groupby('urban')['pc11_pca_no_hh'].sum() / ranchi_khunti_census['pc11_pca_no_hh'].sum()

length of ranchi khunti: 2084
Num shrug rows with no census data: 24
Num shrug rows with no data on urban status: 24


urban
0.0    0.625107
1.0    0.374893
Name: pc11_pca_no_hh, dtype: float64

In [5]:
# create a folium map for the rows with no census data
ranchi_khunti_no_census = ranchi_khunti_census[ranchi_khunti_census['pc11_pca_no_hh'].isna()]

def create_folium_map(gdf):
    # get the center of the delhi map
    center = gdf.centroid.iloc[0]

    # create a folium map showing the villages in the buffer
    m = folium.Map(location=[center.y, center.x], zoom_start=12)
    folium.GeoJson(gdf, tooltip=folium.features.GeoJsonTooltip(fields=['tv_name'])).add_to(m)
    return m

m = create_folium_map(ranchi_khunti_no_census)
m

/var/folders/_b/2wf3gjnx42s4k_vs5n079mh40000gn/T/ipykernel_54241/676096483.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = gdf.centroid.iloc[0]


# Sample SHRUG using PPS
I sample WITH replacement since Ranchi is not broken up by ward so if I sampled without replacement Ranchi would be very underrepresented. 

In [6]:
probabilities = ranchi_khunti_census['pc11_pca_no_hh'] / ranchi_khunti_census['pc11_pca_no_hh'].sum()
sampled_group = ranchi_khunti_census.sample(n=NUM_PSUS_TO_SAMPLE, weights=probabilities, replace=True, random_state=42)
sampled_group

,state_id,district_id,subdistrict_id,village_id,tv_name,geometry,shrid2,state_name,district_name,pc11_pca_no_hh,urban
200,20,364,2684,801794,Ranchi,"POLYGON ((85.38199 23.43523, 85.38327 23.43388...",11-20-364-02684-801794,jharkhand,ranchi,207636.0,1.0
1871,20,365,2704,375236,Chinchal,"POLYGON ((85.461 23.0822, 85.46087 23.08191, 8...",11-20-365-02704-375236,jharkhand,khunti,42.0,0.0
980,20,364,2695,374346,Tero,"POLYGON ((85.00861 23.33943, 85.00877 23.33827...",11-20-364-02695-374346,jharkhand,ranchi,465.0,0.0
692,20,364,2690,374058,Tati,"POLYGON ((85.4482 23.3896, 85.44715 23.38811, ...",11-20-364-02690-374058,jharkhand,ranchi,2544.0,1.0
200,20,364,2684,801794,Ranchi,"POLYGON ((85.38199 23.43523, 85.38327 23.43388...",11-20-364-02684-801794,jharkhand,ranchi,207636.0,1.0
200,20,364,2684,801794,Ranchi,"POLYGON ((85.38199 23.43523, 85.38327 23.43388...",11-20-364-02684-801794,jharkhand,ranchi,207636.0,1.0
115,20,364,2684,373482,Uparkonki,"POLYGON ((85.2784 23.5295, 85.27796 23.52918, ...",11-20-364-02684-373482,jharkhand,ranchi,297.0,0.0
1441,20,365,2700,374806,Mehan,"POLYGON ((85.12808 23.17615, 85.12972 23.17556...",11-20-365-02700-374806,jharkhand,khunti,212.0,0.0
692,20,364,2690,374058,Tati,"POLYGON ((85.4482 23.3896, 85.44715 23.38811, ...",11-20-364-02690-374058,jharkhand,ranchi,2544.0,1.0
903,20,364,2694,374269,Kanjgi,"POLYGON ((84.91115 23.42465, 84.90996 23.42472...",11-20-364-02694-374269,jharkhand,ranchi,132.0,0.0


In [7]:
# drop duplicates
sampled_group_no_dupes = sampled_group.drop_duplicates(subset='shrid2')
sampled_group_no_dupes

,state_id,district_id,subdistrict_id,village_id,tv_name,geometry,shrid2,state_name,district_name,pc11_pca_no_hh,urban
200,20,364,2684,801794,Ranchi,"POLYGON ((85.38199 23.43523, 85.38327 23.43388...",11-20-364-02684-801794,jharkhand,ranchi,207636.0,1.0
1871,20,365,2704,375236,Chinchal,"POLYGON ((85.461 23.0822, 85.46087 23.08191, 8...",11-20-365-02704-375236,jharkhand,khunti,42.0,0.0
980,20,364,2695,374346,Tero,"POLYGON ((85.00861 23.33943, 85.00877 23.33827...",11-20-364-02695-374346,jharkhand,ranchi,465.0,0.0
692,20,364,2690,374058,Tati,"POLYGON ((85.4482 23.3896, 85.44715 23.38811, ...",11-20-364-02690-374058,jharkhand,ranchi,2544.0,1.0
115,20,364,2684,373482,Uparkonki,"POLYGON ((85.2784 23.5295, 85.27796 23.52918, ...",11-20-364-02684-373482,jharkhand,ranchi,297.0,0.0
1441,20,365,2700,374806,Mehan,"POLYGON ((85.12808 23.17615, 85.12972 23.17556...",11-20-365-02700-374806,jharkhand,khunti,212.0,0.0
903,20,364,2694,374269,Kanjgi,"POLYGON ((84.91115 23.42465, 84.90996 23.42472...",11-20-364-02694-374269,jharkhand,ranchi,132.0,0.0


# Get rooftop data for these PSUs

In [8]:
sampled_group_no_dupes['s2_cell_id'] = sampled_group_no_dupes.apply(lambda x: get_s2_cell_id(x.geometry.centroid), axis=1)
sampled_group_no_dupes.head()

/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,state_id,district_id,subdistrict_id,village_id,tv_name,geometry,shrid2,state_name,district_name,pc11_pca_no_hh,urban,s2_cell_id
200,20,364,2684,801794,Ranchi,"POLYGON ((85.38199 23.43523, 85.38327 23.43388...",11-20-364-02684-801794,jharkhand,ranchi,207636.0,1.0,4176244229456003072
1871,20,365,2704,375236,Chinchal,"POLYGON ((85.461 23.0822, 85.46087 23.08191, 8...",11-20-365-02704-375236,jharkhand,khunti,42.0,0.0,4176244229456003072
980,20,364,2695,374346,Tero,"POLYGON ((85.00861 23.33943, 85.00877 23.33827...",11-20-364-02695-374346,jharkhand,ranchi,465.0,0.0,4176244229456003072
692,20,364,2690,374058,Tati,"POLYGON ((85.4482 23.3896, 85.44715 23.38811, ...",11-20-364-02690-374058,jharkhand,ranchi,2544.0,1.0,4176244229456003072
115,20,364,2684,373482,Uparkonki,"POLYGON ((85.2784 23.5295, 85.27796 23.52918, ...",11-20-364-02684-373482,jharkhand,ranchi,297.0,0.0,4176244229456003072


In [9]:
# create empty gdf to store rooftops
all_rooftops = gpd.GeoDataFrame()

# since all the PSUs have the same s2_cell_id we can just use the first one
s2_cell_id = sampled_group_no_dupes['s2_cell_id'].iloc[0]

rooftops_gdf = gpd.read_parquet(ROOFTOP_DIR /f'{s2_cell_id}.parquet')


In [10]:
temp_rooftops = gpd.GeoDataFrame()
all_rooftops = gpd.GeoDataFrame()

for item, row in sampled_group_no_dupes.iterrows():
    # print a single dot without the newline character
    print('.', end='')
    # filter rooftops to only include those within the psu
    temp_rooftops = rooftops_gdf[rooftops_gdf.geometry.within(row.geometry)]
    # set the psid for the rooftops
    temp_rooftops['shrid2'] = row['shrid2']
    temp_rooftops['urban'] = row['urban']
    temp_rooftops.to_crs(epsg=4326, inplace=True)
    # try to append temp_rooftops to all_rooftops and catch a value error. if there is an error, print the crs of the two dataframes
    try:
        all_rooftops = gpd.GeoDataFrame(pd.concat([all_rooftops, temp_rooftops], ignore_index=True))
    except ValueError:
        print(f"Error with s2 cell {s2_cell_id}")
        print(f"temp_rooftops crs: {temp_rooftops.crs}")
        print(f"all_rooftops crs: {all_rooftops.crs}")

.

/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


.

/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


.

...

.

/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


.

/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


.

In [11]:
# save all rooftops to a parquet file in case I close this notebook or the kernel gets messed up
all_rooftops.to_parquet(OUTPUT_DIR / f'rooftops_sampled_psus_ranchi_khunti.parquet')

# Sample 4 rooftops from each PSU (except Ranchi where I sample 12)

In [12]:
all_rooftops['neighbors'] = count_neighbors_in_radius(all_rooftops)
all_rooftops['isolated'] = (all_rooftops['neighbors'] < 10)

# create dataframe of all non-isolated rooftops
non_isolated_rooftops = all_rooftops[~all_rooftops['isolated']]

In [13]:
ranchi_rooftops = non_isolated_rooftops[non_isolated_rooftops['shrid2'].str.contains('11-20-364-02684-801794')]
other_rooftops =   non_isolated_rooftops[~non_isolated_rooftops['shrid2'].str.contains('11-20-364-02684-801794')]

In [14]:
# sample 4 rooftops in each PSU for other rooftops
sampled_points_other = other_rooftops.groupby('shrid2', group_keys=False).apply(lambda x: x.sample(n=min(4, x.shape[0]),random_state=42))

# sample 12 rooftops in ranchi_rooftops
sampled_points_ranchi = ranchi_rooftops.sample(n=12, random_state=42)

/var/folders/_b/2wf3gjnx42s4k_vs5n079mh40000gn/T/ipykernel_54241/96762827.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_points_other = other_rooftops.groupby('shrid2', group_keys=False).apply(lambda x: x.sample(n=min(4, x.shape[0]),random_state=42))


In [15]:
# concat the two dataframes
sampled_points = pd.concat([sampled_points_other, sampled_points_ranchi])

# get the centroid of each rooftop
sampled_points['geometry'] = sampled_points.geometry.centroid

/var/folders/_b/2wf3gjnx42s4k_vs5n079mh40000gn/T/ipykernel_54241/3276395114.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  sampled_points['geometry'] = sampled_points.geometry.centroid


# Get nearest points on road 

In [16]:
# for each point in the sampled points, get the nearest point on the road
sampled_points['nearest_point_on_road'] = sampled_points.progress_apply(lambda x: get_nearest_point_on_road(x.geometry), axis=1)

100%|██████████| 32/32 [00:04<00:00,  7.79it/s]


# Manual revision of points

Some of the rural points that Google says lie on a road don't really seem to be on a road. For these, I set the "nearest_point_on_road" to null. I use the geohash for this because checking for equality of lat and long is really annoying because Google maps truncates the precision of the point.

In [54]:
points_not_really_on_road = [
    'tuhky4j4',
    'tuhkwntc',
    'tuhec1j9',
    'tuhg23zu',
    'tuhe9p4w',
    'tuhg298p',
    'tuhg2992' 
]

# for points that are not really on the road, set nearest_point_on_road to None
sampled_points.loc[sampled_points['geohash'].isin(points_not_really_on_road), 'nearest_point_on_road'] = None

# remove one point that is in the airport
sampled_points = sampled_points[sampled_points['geohash'] != 'tuhstt20']

7

In [17]:
sampled_points['distance_to_road_m'] = sampled_points.apply(lambda x: dist_in_meters(x.geometry, x.nearest_point_on_road), axis=1)

# create binary variable called "on_road" that is 1 if nearest_point_on_road is not null
sampled_points['on_road'] = sampled_points['nearest_point_on_road'].notnull().astype(int)

# Display map of points

In [18]:
# create folium map of points
center = sampled_points.geometry.iloc[0]

# create a folium map showing the villages in the buffer
m = folium.Map(location=[center.y, center.x], zoom_start=12, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr='Esri')

# Function to determine marker color based on 'urban' and 'on_road' values
def get_marker_color(urban, on_road):
    if urban and on_road:
        return 'red'
    elif urban and not on_road:
        return 'blue'
    elif not urban and on_road:
        return 'green'
    else:
        return 'gray'

# Iterate through the GeoDataFrame and add points to the map
for idx, row in sampled_points.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=f"Urban: {row['urban']}, On Road: {row['on_road']}",
        icon=folium.Icon(color=get_marker_color(row['urban'], row['on_road']))
    ).add_to(m)

# save the map as sampled points
m.save(OUTPUT_DIR / 'sampled_points.html')
m

# Save sample outputs as CSV

In [19]:
temp_samp_points = sampled_points[['shrid2', 'geometry', 'nearest_point_on_road', 'distance_to_road_m', 'urban']]
temp_samp_points['orig_lat'] = temp_samp_points['geometry'].y
temp_samp_points['orig_lon'] = temp_samp_points['geometry'].x
temp_samp_points['road_lat'] = temp_samp_points['nearest_point_on_road'].y
temp_samp_points['road_lon'] = temp_samp_points['nearest_point_on_road'].x
temp_samp_points = temp_samp_points.drop(columns=['geometry', 'nearest_point_on_road'])
temp_samp_points['google_maps_directions_link'] = temp_samp_points.apply(
    lambda row: f"https://www.google.com/maps/dir/{row['orig_lat']},{row['orig_lon']}/{row['road_lat']},{row['road_lon']}",
    axis=1
)
# replace google_maps_directions_link with a link to just the original point if nearest_point_on_road is null
temp_samp_points.loc[temp_samp_points['road_lat'].isna(), 'google_maps_directions_link'] = temp_samp_points.loc[
    temp_samp_points['road_lat'].isna(), ['orig_lat', 'orig_lon']
].apply(lambda x: f"https://www.google.com/maps?q={x[0]},{x[1]}", axis=1)


# save the sampled points as csv
temp_samp_points.to_csv(OUTPUT_DIR / f'samp_points.csv', index=False)

/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodatafr

# Save points as kmls
I create several different kml files:

1. a kml files of lines between the original point and the nearest point on road
2. a kml file of points on road that are urban
3. a kml file of points on road that are rural
4. a kml file of points NOT on road (all are rural)

I then upload each of these kml files to a single Google My Map separately. While possible to combine all of these in python it is a bit of a pain.


In [20]:
sampled_points_on_road = sampled_points[~(sampled_points['nearest_point_on_road'].isnull())]
sampled_points_on_road_urban = sampled_points_on_road[sampled_points_on_road['urban'] == 1]
sampled_points_on_road_rural = sampled_points_on_road[sampled_points_on_road['urban'] == 0]
sampled_points_off_road = sampled_points[sampled_points['nearest_point_on_road'].isnull()]

# save lines between original points and nearest points on road
sampled_points_on_road['line'] = sampled_points_on_road.apply(lambda row: LineString([row['geometry'], row['nearest_point_on_road']]), axis=1)
lines_gdf = sampled_points_on_road[['line', 'shrid2']]
lines_gdf.to_file(OUTPUT_DIR / 'lines.kml', driver='KML')

# save sampled points on road and urban as kml
sampled_points_on_road_urban[['nearest_point_on_road', 'shrid2']].to_file(OUTPUT_DIR / 'points_on_road_urban.kml', driver='KML')

# save sampled points on road and rural as kml
sampled_points_on_road_rural[['nearest_point_on_road', 'shrid2']].to_file(OUTPUT_DIR / 'points_on_road_rural.kml', driver='KML')

# save sampled points off road as kml
sampled_points_off_road[['geometry', 'shrid2']].to_file(OUTPUT_DIR / 'points_off_road.kml', driver='KML')


/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
/Users/douglasjohnson/Documents/code/pin_drop_sampling2/.venv/lib/python3.11/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
/Users/douglasjohns

In [21]:
sampled_points_on_road_rural.head(10)

,boundary_id,bf_source,confidence,area_in_meters,s2_id,country_iso,geometry,geohash,shrid2,urban,neighbors,isolated,nearest_point_on_road,distance_to_road_m,on_road
347252,56,google,0.8294,32.1143,4176244229456003072,IND,POINT (85.2714 23.5167),tuhtu3hd,11-20-364-02684-373482,0.0,54,False,POINT (85.27137 23.51662),8.897919,1
346684,56,google,0.7639,18.1215,4176244229456003072,IND,POINT (85.26911 23.51335),tuhtu27p,11-20-364-02684-373482,0.0,10,False,POINT (85.26904 23.5135),18.728227,1
346934,56,google,0.8076,44.6793,4176244229456003072,IND,POINT (85.27203 23.51619),tuhtu2vp,11-20-364-02684-373482,0.0,53,False,POINT (85.2721 23.51617),6.582941,1
346857,56,google,0.8746,61.9966,4176244229456003072,IND,POINT (85.27058 23.51617),tuhtu2up,11-20-364-02684-373482,0.0,59,False,POINT (85.27071 23.51613),14.341226,1
342876,56,google,0.6514,63.0674,4176244229456003072,IND,POINT (85.00041 23.33629),tuhky0pr,11-20-364-02695-374346,0.0,20,False,POINT (85.00047 23.33629),6.287872,1
343383,56,microsoft,NaN,41.5028,4176244229456003072,IND,POINT (84.99741 23.34643),tuhky4j4,11-20-364-02695-374346,0.0,36,False,POINT (84.99729 23.34642),12.322526,1
342153,56,google,0.8603,120.2944,4176244229456003072,IND,POINT (84.99832 23.32699),tuhkwntc,11-20-364-02695-374346,0.0,10,False,POINT (84.99804 23.32707),29.628141,1
342179,56,microsoft,NaN,24.6467,4176244229456003072,IND,POINT (84.99538 23.33341),tuhkwpew,11-20-364-02695-374346,0.0,16,False,POINT (84.99538 23.33335),6.574817,1
347853,56,google,0.8447,241.2210,4176244229456003072,IND,POINT (85.1297 23.16495),tuhec1j9,11-20-365-02700-374806,0.0,43,False,POINT (85.12981 23.1649),12.467627,1
347564,56,google,0.8558,164.8961,4176244229456003072,IND,POINT (85.12572 23.15484),tuhe9p4w,11-20-365-02700-374806,0.0,40,False,POINT (85.12578 23.15483),7.074052,1
